In [ ]:
!pip install fer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
project_path = "/content/drive/My Drive/ml/edyo/project"

In [21]:
import logging

logger = logging.getLogger(__name__)
debug_handler = logging.FileHandler(f"{project_path}/output/emotions/output.log", mode='w')
debug_handler.setLevel(logging.DEBUG)

debug_handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(debug_handler)

In [14]:
from fer import FER
from collections import Counter

import cv2
import numpy as np

import time
import os

In [15]:
detector = FER(mtcnn=True)
detector._get_labels()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


{0: 'angry',
 1: 'disgust',
 2: 'fear',
 3: 'happy',
 4: 'sad',
 5: 'surprise',
 6: 'neutral'}

In [26]:
def draw_box(image_src, bbox, text=None):
    image = image_src.copy()
    x, y, w, h = bbox

    color = (0, 0, 255)
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 3)

    if text is not None:
        if image.shape[0] > y+h:
            cv2.rectangle(image, (x, y+h-30), (x+w, y+h), color, -1)
            cv2.putText(image, text, (x+6, y+h-10), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (255, 255, 255), 1, cv2.LINE_AA) 
        else:
            cv2.rectangle(image, (x, y-30), (x+w, y), color, -1)
            cv2.putText(image, text, (x+6, y-10), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (255, 255, 255), 1, cv2.LINE_AA) 
        
    return image

def process_video(video_path, output_folder, skip_frames=10):
    video_name = os.path.basename(video_path)
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    i = 0
    emotions = []

    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    w, h = frame.shape[:2]
    out = cv2.VideoWriter(f'{output_folder}/fer_{video_name}', fourcc, fps, (h, w))

    process_time = 0

    while(ret):
        frame = frame[:,:,::-1].copy()

        if i==0 or i%skip_frames==0:
            start_time = time.time()
            emotions_result = detector.detect_emotions(frame.copy())
        #     print(emotions_result)
        #     emotion, score = detector.top_emotion(frame.copy()) # 'happy', 0.99
            process_time += time.time() - start_time
        image = frame.copy()
        for er in emotions_result:
            emotion = max(er['emotions'], key=er['emotions'].get)
            image = draw_box(image, er['box'], text=emotion)
            emotions.append(emotion)
            
        out.write(image[:,:,::-1])
        ret, frame = cap.read()
        i += 1

    cap.release()
    out.release()

    logger.debug(f"Video name: {video_name}")
    logger.debug(f"Input video fps: {fps}")
    logger.debug(f"Process time: {process_time:.2}s ({i/process_time:.2f} fps)")
    logger.debug(f"Skip frames: {skip_frames}")
    # logger.debug(f"Processed frames: {i}")
    # logger.debug(f"Emotions detected: {len(emotions)}")
    logger.debug(Counter(emotions))
    logger.debug("===")

In [27]:
video_list = ["video_1_0000.mp4", "video_2_0000.mp4", "video_3_0001.mp4", "video_3_0045.mp4", "video_4_0001.mp4"]

for video_name in video_list:
    process_video(f"{project_path}/dataset/video/medium/{video_name}", f"{project_path}/output/emotions", 25)